In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd

bgg_Category = pd.read_csv('scraped/bgg_Category.csv')
bgg_GameFamily = pd.read_csv('scraped/bgg_GameFamily.csv')
bgg_GameItem = pd.read_csv('scraped/bgg_GameItem.csv')
bgg_GameType = pd.read_csv('scraped/bgg_GameType.csv')
bgg_Mechanic = pd.read_csv('scraped/bgg_Mechanic.csv')
bgg_Person = pd.read_csv('scraped/bgg_Person.csv')
bgg_Publisher = pd.read_csv('scraped/bgg_Publisher.csv')

In [2]:
drop=[
    'bga_id',
'dbpedia_id',
'luding_id',
'spielen_id',
'wikidata_id',
'wikipedia_id']

<h1>Cleaning

In [3]:
#df_core = bgg_GameItem.copy()[core+game_covariates+response]
df_core = bgg_GameItem.drop(columns=drop).copy()
core = df_core[df_core.avg_rating.notna()] # at least 1 ratings
core = core[core.bayes_rating.notna()] # at least 30 ratings
core = core[(core.year >= 1990) & (core.year < 2024)] # only games after key:  1990 <= year < 2024
core.designer = core.designer.str.split(",")
core [core.designer.map(lambda x: "3" in x, na_action="ignore").fillna(False)] = pd.NA # 3 is designer="uncredited"
core = core[core.designer.notna()] # must include designer
core = core[core.mechanic.notna()] # must include mechanics
core.mechanic = core.mechanic.str.split(",").map(set).map(sorted).map(tuple)
core = core.sort_values("year").reset_index(drop=True)
core.head(2)

,bgg_id,name,year,game_type,designer,artist,publisher,min_players,max_players,min_players_rec,...,family,implementation,integration,rank,num_votes,avg_rating,stddev_rating,bayes_rating,complexity,language_dependency
0,299.0,De Bellis Antiquitatis: Quick Play Wargame Rul...,1990.0,4664,"[1668, 2014, 1669]","1333,24542","257,938,44625",2.0,6.0,2.0,...,"52373,5606,6324,72016",NaN,NaN,3160.0,686.0,7.29609,1.64853,5.93811,2.8,NaN
1,8406.0,Operation Solace,1990.0,4664,"[1747, 2868, 1718]",NaN,"3133,189",2.0,2.0,2.0,...,"9470,10618,21633,61979,26464",NaN,NaN,19455.0,48.0,5.95521,1.31716,5.50717,3.2,NaN


In [4]:
by_id = core.sort_values("year").reset_index(drop=True)
by_id.designer = by_id.designer.str.split(",")
by_id = by_id.explode("designer")
by_id["event_type"] = "event"
main = ["bgg_id","year","designer","event_type"]
by_id = by_id[main+[x for x in by_id if x not in main]]
by_id.head(2)

,bgg_id,year,designer,event_type,name,game_type,artist,publisher,min_players,max_players,...,family,implementation,integration,rank,num_votes,avg_rating,stddev_rating,bayes_rating,complexity,language_dependency
0,299.0,1990.0,NaN,event,De Bellis Antiquitatis: Quick Play Wargame Rul...,4664,"1333,24542","257,938,44625",2.0,6.0,...,"52373,5606,6324,72016",NaN,NaN,3160.0,686.0,7.29609,1.64853,5.93811,2.8,NaN
1,7994.0,1990.0,NaN,event,"Silver Bayonet: The First Team in Vietnam, 1965",4664,"2652,71,19820,1920,140",52,2.0,2.0,...,"10618,77911,61979,72016,3337",NaN,NaN,9331.0,151.0,6.68742,1.52534,5.57696,3.0,NaN


<h1>Compute Novelties

In [5]:
#step 1
configurations = core.mechanic.unique()
lut = {v:k for k,v in enumerate(configurations)}
dummies = pd.Series(configurations).explode().str.get_dummies().groupby(level=0).sum()
dummies 

,2001,2002,2003,2004,2005,2007,2008,2009,2010,2011,...,3007,3096,3097,3098,3099,3100,3101,3102,3103,3104
0,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9453,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9454,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9455,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9456,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [6]:
#step 2: euclidean distances between each pair of configurations
from sklearn.metrics import pairwise_distances
dist_matrix = pd.DataFrame(pairwise_distances(dummies),index=dummies.index,columns=dummies.index)
dist_matrix = dist_matrix/dist_matrix.max()
dist_matrix

,0,1,2,3,4,5,6,7,8,9,...,9448,9449,9450,9451,9452,9453,9454,9455,9456,9457
0,0.000000,0.559017,0.516398,0.447214,0.516398,0.491304,0.447214,0.491304,0.538816,0.415227,...,0.594089,0.594089,0.538816,0.557086,0.560612,0.595683,0.612372,0.485071,0.557086,0.612372
1,0.542326,0.000000,0.365148,0.447214,0.577350,0.557086,0.516398,0.557086,0.595683,0.491304,...,0.641689,0.594089,0.595683,0.615882,0.609449,0.595683,0.661438,0.594089,0.669534,0.661438
2,0.485071,0.353553,0.000000,0.365148,0.516398,0.491304,0.447214,0.491304,0.538816,0.415227,...,0.594089,0.594089,0.538816,0.557086,0.560612,0.538816,0.612372,0.594089,0.615882,0.612372
3,0.420084,0.433013,0.365148,0.000000,0.365148,0.321634,0.365148,0.321634,0.401610,0.321634,...,0.485071,0.485071,0.401610,0.491304,0.507093,0.538816,0.559017,0.542326,0.557086,0.500000
4,0.485071,0.559017,0.516398,0.365148,0.000000,0.321634,0.365148,0.321634,0.401610,0.321634,...,0.485071,0.485071,0.401610,0.491304,0.507093,0.538816,0.559017,0.542326,0.557086,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9453,0.568796,0.586302,0.547723,0.547723,0.547723,0.525226,0.483046,0.525226,0.567962,0.454859,...,0.568796,0.514496,0.508001,0.525226,0.632456,0.000000,0.586302,0.618347,0.587220,0.530330
9454,0.594089,0.661438,0.632456,0.577350,0.577350,0.557086,0.516398,0.557086,0.595683,0.491304,...,0.641689,0.641689,0.595683,0.557086,0.654654,0.595683,0.000000,0.641689,0.615882,0.612372
9455,0.485071,0.612372,0.632456,0.577350,0.577350,0.491304,0.516398,0.557086,0.595683,0.491304,...,0.641689,0.594089,0.595683,0.557086,0.560612,0.647576,0.661438,0.000000,0.669534,0.661438
9456,0.514496,0.637377,0.605530,0.547723,0.547723,0.525226,0.483046,0.454859,0.567962,0.454859,...,0.618347,0.568796,0.567962,0.525226,0.534522,0.567962,0.586302,0.618347,0.000000,0.530330


In [7]:
#step 3: novelty = average of distances of the focal game to all of the games that existed at that time

#questions that arise: level of detail => yearly => creates ambiguity => check stability
from tqdm import tqdm
import numpy as np

mechanics_by_year = core[["bgg_id","year","mechanic"]].copy().reset_index(drop=True)
mechanics_by_year["configuration_id"] = mechanics_by_year.mechanic.map(lambda x :lut[x])
mechanics_by_year

permutations = 5
novelty = pd.DataFrame()
for _ in range(permutations):
    permutation = mechanics_by_year.groupby("year").transform(np.random.permutation).reset_index(drop=True)
    i = 0
    for ix in tqdm(permutation.index):
        if ix == 0:
            permutation["novelty"] = pd.NA
        else:
            focal = permutation.loc[ix,"configuration_id"]
            previous = permutation.loc[:ix-1,"configuration_id"]
            permutation.loc[ix,"novelty"] = dist_matrix.to_numpy()[previous,focal].mean()
    novelty = pd.concat((novelty,permutation.set_index("bgg_id")[["novelty"]]),axis=1)

100%|██████████| 19803/19803 [00:01<00:00, 9904.16it/s] 


In [8]:
def min_max(df):
    return (df-df.min())/(df.max()-df.min())

novelty_scores = pd.concat((novelty.mean(axis=1).rename("novelty_avg"),novelty.std(axis=1).rename("novelty_std")),axis=1)
novelty_scores["novelty_norm"] = novelty_scores[["novelty_avg"]].apply(min_max).convert_dtypes()
core = core.set_index("bgg_id").join(novelty_scores).reset_index()
novelty_scores

,novelty_avg,novelty_std,novelty_norm
bgg_id,,,
2492.0,0.41292,0.077456,0.362478
5108.0,0.410159,0.140257,0.352466
6020.0,0.392364,0.062958,0.287936
1503.0,0.366176,0.082949,0.192968
1940.0,0.408923,0.116757,0.347983
...,...,...,...
359974.0,0.440162,0.056463,0.461266
379005.0,0.437628,0.053321,0.452077
390092.0,0.436832,0.037516,0.449191


<h1> Add dummy events and export


In [10]:
add_actor = core.explode(column="designer").groupby("designer")["year"].min().reset_index()
add_actor["event_type"]= "add_actor"
add_actor.head(2)

,designer,year,event_type
0,1,1990.0,add_actor
1,10,1996.0,add_actor


In [11]:
core["event_type"] = "event"
eventlist = pd.concat([core,add_actor])
eventlist = eventlist.sort_values(["year","event_type"],ascending=(True,True)).reset_index(drop=True).convert_dtypes()
numeric = eventlist.select_dtypes("number").columns 
eventlist.loc[:,numeric] = eventlist.loc[:,numeric].fillna(0)
eventlist = eventlist.explode(column="designer")
eventlist

,bgg_id,name,year,game_type,designer,artist,publisher,min_players,max_players,min_players_rec,...,num_votes,avg_rating,stddev_rating,bayes_rating,complexity,language_dependency,novelty_avg,novelty_std,novelty_norm,event_type
0,0,<NA>,1990,<NA>,1,<NA>,<NA>,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,add_actor
1,0,<NA>,1990,<NA>,1002,<NA>,<NA>,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,add_actor
2,0,<NA>,1990,<NA>,103,<NA>,<NA>,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,add_actor
3,0,<NA>,1990,<NA>,1047,<NA>,<NA>,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,add_actor
4,0,<NA>,1990,<NA>,1050,<NA>,<NA>,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,add_actor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29674,332804,Cantaloop: Book 3 – Against All Odds,2023,<NA>,34802,"132194,11507",234,1,4,1,...,54,7.63148,1.25828,5.55754,0.0,0.0,0.442809,0.083918,0.470866,event
29675,364277,Stonks,2023,<NA>,79512,104561,"42325,35035",2,5,2,...,45,7.59356,1.72535,5.53414,1.8333,0.0,0.493527,0.065905,0.654788,event
29675,364277,Stonks,2023,<NA>,14650,104561,"42325,35035",2,5,2,...,45,7.59356,1.72535,5.53414,1.8333,0.0,0.493527,0.065905,0.654788,event
29676,282775,The Warp,2023,5497,118938,72115,"25996,42144,5380",1,4,1,...,179,8.37989,1.38852,5.66547,3.3077,0.0,0.470497,0.051881,0.571274,event


In [14]:
predictor = [
'year',
#'name',
'bgg_id',
'designer',
]
response = [
'num_votes',
'avg_rating',
#'stddev_rating',
#'rank',
'bayes_rating',
"novelty_norm",
"novelty_avg",
]
actor_covariates = [

]
game_covariates = [
# 'mechanic',
 'complexity',
 'game_type',
# 'publisher',
 'min_players',
 'max_players',
# 'min_players_rec',
# 'max_players_rec',
# 'min_players_best',
# 'max_players_best',
# 'min_age',
# 'min_age_rec',
# 'min_time',
# 'max_time',
 'category',
# 'cooperative',
# 'family',
# 'language_dependency',
# 'artist',
# # filter out ?
# 'compilation',
# 'compilation_of',
# 'implementation',
# 'integration',
#
]

In [15]:
import csv
cols = predictor + response + game_covariates + actor_covariates + ["event_type"]
eventlist[cols].to_csv("../bgg_min.csv",index=False,quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
# dummy events for all actor variables
